# Etapa 1

Montar o Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Etapa 2

Definir os caminhos dos arquivos

In [114]:
import os
import glob

pasta_input = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input'

# Busca o arquivo XLSX
arquivo_xlsx = None  # Inicializa arquivo_xlsx
arquivos_xlsx = glob.glob(os.path.join(pasta_input, '*.xlsx'))
if arquivos_xlsx:
    arquivo_xlsx = arquivos_xlsx[0]  # Pega o primeiro arquivo XLSX encontrado
    print(f'Arquivo XLSX encontrado: {arquivo_xlsx}')
else:
    print('Nenhum arquivo XLSX encontrado na pasta.')

# Busca todos os arquivos CSV
arquivos_csv = glob.glob(os.path.join(pasta_input, '*.csv'))
if arquivos_csv:
    print(f'Arquivos CSV encontrados:')
    for arquivo_csv in arquivos_csv:
        print(f'  - {arquivo_csv}')
else:
    print('Nenhum arquivo CSV encontrado na pasta.')

Arquivo XLSX encontrado: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input/Cartao Nubank.xlsx
Arquivos CSV encontrados:
  - /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input/Nubank_2025-07-12.csv


# Etapa 3

Carregar o arquivo XLSX para obter o cabeçalho

In [115]:
import pandas as pd

caminho_xlsx = os.path.join(pasta_input, arquivo_xlsx)
try:
    df_xlsx = pd.read_excel(caminho_xlsx)
    header_base = list(df_xlsx.columns)
    print(f"Arquivo XLSX '{arquivo_xlsx}' lido para extrair o cabeçalho.")
except Exception as e:
    print(f"Erro ao ler arquivo XLSX '{arquivo_xlsx}': {e}")
    exit()


Arquivo XLSX '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input/Cartao Nubank.xlsx' lido para extrair o cabeçalho.


# Etapa 4

Processar os arquivos CSV

In [126]:
# Célula 5: Processar os arquivos CSV
lista_dfs_csv = []
for arquivo_csv in arquivos_csv:
    caminho_csv = os.path.join(pasta_input, arquivo_csv)
    try:
        df_csv = pd.read_csv(caminho_csv)
        # Define as colunas desejadas
        colunas_desejadas = ['date', 'category', 'title', 'amount'] # 'category' agora está na lista
        df_csv['category'] = None  # Cria a coluna 'category' com valor None
        df_csv = df_csv[['date', 'title', 'amount']].copy()  # Seleciona e copia as colunas 'date', 'title', 'amount'
        df_csv.insert(1, 'category', None) # Insere a coluna 'category' na posição 1 (entre 'date' e 'title')

        # Renomeia a coluna 'date' para 'Data' para corresponder ao XLSX
        df_csv = df_csv.rename(columns={'date': 'Data'})

        print(f"Lendo arquivo CSV: {arquivo_csv}")
        print(f"Shape do DataFrame do CSV: {df_csv.shape}")

        lista_dfs_csv.append(df_csv)
        print(f"Arquivo CSV '{arquivo_csv}' processado com sucesso.")
    except (KeyError, FileNotFoundError, pd.errors.EmptyDataError, Exception) as e:
        # Captura erros comuns ao ler CSVs e outros erros
        print(f"Erro ao processar arquivo CSV '{arquivo_csv}': {e}")
        continue

# Concatena os DataFrames CSV após o loop
if lista_dfs_csv:
    df_final = pd.concat(lista_dfs_csv, ignore_index=True)
    print("Arquivos CSV concatenados com sucesso.")
else:
    print("Nenhum arquivo CSV foi processado.")
    df_final = pd.DataFrame()


Lendo arquivo CSV: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input/Nubank_2025-07-12.csv
Shape do DataFrame do CSV: (2, 4)
Arquivo CSV '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input/Nubank_2025-07-12.csv' processado com sucesso.
Arquivos CSV concatenados com sucesso.


# Etapa 5

Concatenar os DataFrames df_xlsx e df_final

In [128]:
if lista_dfs_csv: # Verifica se lista_dfs_csv não está vazia
    df_final = pd.concat(lista_dfs_csv, ignore_index=True)
    print("Arquivos CSV concatenados com sucesso.")
    df_concatenado = pd.concat([df_xlsx, df_final], ignore_index=True)
    print("DataFrame XLSX e DataFrames CSV concatenados com sucesso.")

    # Remove linhas em branco (onde todos os valores são NaN)
    df_concatenado = df_concatenado.dropna(how='all')
    print("Linhas em branco removidas.")

    # Formata a coluna 'Data' para o formato 'AAAA-MM-DD'
    df_concatenado['Data'] = pd.to_datetime(df_concatenado['Data']).dt.strftime('%Y-%m-%d')
    df_concatenado['Data Vencimento'] = pd.to_datetime(df_concatenado['Data Vencimento']).dt.strftime('%Y-%m-%d')
    print("Coluna 'Data' formatada para AAAA-MM-DD.")

    print("Primeiras linhas do DataFrame concatenado:")
    print(df_concatenado.head())  # Exibe as primeiras linhas para visualização
else:
    print("Nenhum arquivo CSV foi processado. Apenas o DataFrame XLSX será utilizado.")
    df_concatenado = df_xlsx # Atribui df_xlsx a df_concatenado para evitar erros nas próximas células
    df_concatenado = df_concatenado.dropna(how='all') # Remove linhas em branco do df_concatenado
    # Formata a coluna 'Data' para o formato 'AAAA-MM-DD'
    df_concatenado['Data'] = pd.to_datetime(df_concatenado['Data']).dt.strftime('%Y-%m-%d')
    df_concatenado['Data Vencimento'] = pd.to_datetime(df_concatenado['Data Vencimento']).dt.strftime('%Y-%m-%d')
    print("Coluna 'Data' formatada para AAAA-MM-DD.")
    print("Primeiras linhas do DataFrame concatenado:")
    print(df_concatenado.head())


Arquivos CSV concatenados com sucesso.
DataFrame XLSX e DataFrames CSV concatenados com sucesso.
Linhas em branco removidas.
Coluna 'Data' formatada para AAAA-MM-DD.
Primeiras linhas do DataFrame concatenado:
         Data   category             Estabelecimento   Valor Data Vencimento  \
0  2022-11-28  vestuário            Yan Calcados 2/3   50.00      2023-01-05   
1  2022-11-28  vestuário        Cea Gru 265 Ecpc 2/3  223.97      2023-01-05   
2  2022-11-28  vestuário      Pag*Jsgoiscalcados 2/2   39.99      2023-01-05   
3  2022-11-28   educação  Pg *Power Bi Pbiexperi 2/5   74.95      2023-01-05   
4  2022-11-28   educação         Pg *Curso Beta 9/12   59.70      2023-01-05   

  idCategoria       Descrição Transferencia title  amount  
0       Ves02        calçados           NaN   NaN     NaN  
1       Ves03          roupas           NaN   NaN     NaN  
2       Ves02        calçados           NaN   NaN     NaN  
3       Edu03  curso power bi           NaN   NaN     NaN  
4       Ed

# Etapa 6

Salvar DataFrame em arquivo XLSX

In [129]:
# Salva o DataFrame em um arquivo XLSX, substituindo o original
caminho_arquivo_saida = arquivo_xlsx  # Usa o caminho do arquivo XLSX original
df_concatenado.to_excel(caminho_arquivo_saida, index=False)
print(f"DataFrame salvo com sucesso em: {caminho_arquivo_saida}, substituindo o arquivo original.")


DataFrame salvo com sucesso em: /content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input/Cartao Nubank.xlsx, substituindo o arquivo original.


In [88]:
import pandas as pd
import os

# Obter lista de arquivos na pasta com a extensão .xls ou .xlsx
arquivos = [f for f in os.listdir(pasta_input) if f.endswith('.xls') or f.endswith('.xlsx')]

# Lista para armazenar os DataFrames processados
df_list = []

# Lista fixa com os nomes das colunas
nomes_colunas = [
    'data_lanc', 'categoria', 'descricao', 'valor', 'data_vencimento', 'idcategoria', 'observacao', 'transferencia'
]

# Função para aplicar os nomes fixos nas colunas
def aplicar_nomes_colunas(df, novos_nomes):
    if len(novos_nomes) == len(df.columns):
        df.columns = novos_nomes
    else:
        print(f"Erro: A quantidade de novos nomes não corresponde ao número de colunas em {arquivo}.")
    return df

# Iterar sobre os arquivos da pasta
for arquivo in arquivos:
    # Caminho completo do arquivo
    file_path = os.path.join(pasta_input, arquivo)

    # Ler o arquivo Excel (independente de ser .xls ou .xlsx)
    df = pd.read_excel(file_path, header=None)  # Não utiliza um cabeçalho por enquanto

    # Encontrar a linha que contém o cabeçalho (a que tem 'data' na primeira coluna)
    header_row = df[df.iloc[:, 0].str.contains('data', case=False, na=False)].index[0]

    # Ler novamente o arquivo, agora utilizando o cabeçalho correto
    df = pd.read_excel(file_path)
    #df = pd.read_excel(file_path, header=header_row)

    # Aplicar os nomes fixos nas colunas
    df = aplicar_nomes_colunas(df, nomes_colunas)

    # Adicionar uma coluna com o nome do arquivo
    df['nome_arquivo'] = arquivo

    # Adicionar o DataFrame à lista
    df_list.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_combined = pd.concat(df_list, ignore_index=True)

# Ordenar o DataFrame pela coluna 'data'
df_combined['data_lanc'] = pd.to_datetime(df_combined['data_lanc'], errors='coerce', dayfirst=True)  # Converter a coluna 'data' para datetime
df_combined = df_combined.sort_values(by='data_lanc')  # Ordenar pela coluna 'data'

# Adicionar uma coluna 'data_carga' com a data atual
df_combined['data_carga'] = pd.to_datetime('today').normalize()  # 'normalize' para deixar a hora como 00:00:00

colunas_finais = ['data_lanc', 'categoria', 'descricao', 'valor', 'data_vencimento', 'idcategoria', 'observacao', 'nome_arquivo', 'data_carga']

df_filtrado = df_combined[colunas_finais]

# Exibir as primeiras linhas do DataFrame combinado com a nova coluna
#df_filtrado.head()

# exibir a quantidade de linhas para cada arquivo
# print(df_filtrado.groupby('nome_arquivo').size())


<ipython-input-88-fb524c387349>:51: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_combined['data_lanc'] = pd.to_datetime(df_combined['data_lanc'], errors='coerce', dayfirst=True)  # Converter a coluna 'data' para datetime


In [89]:
df_filtrado

,data_lanc,categoria,descricao,valor,data_vencimento,idcategoria,observacao,nome_arquivo,data_carga
0,2022-11-28,vestuário,Yan Calcados 2/3,50.00,2023-01-05,Ves02,calçados,Cartao Nubank.xlsx,2025-05-10
1,2022-11-28,vestuário,Cea Gru 265 Ecpc 2/3,223.97,2023-01-05,Ves03,roupas,Cartao Nubank.xlsx,2025-05-10
2,2022-11-28,vestuário,Pag*Jsgoiscalcados 2/2,39.99,2023-01-05,Ves02,calçados,Cartao Nubank.xlsx,2025-05-10
3,2022-11-28,educação,Pg *Power Bi Pbiexperi 2/5,74.95,2023-01-05,Edu03,curso power bi,Cartao Nubank.xlsx,2025-05-10
4,2022-11-28,educação,Pg *Curso Beta 9/12,59.70,2023-01-05,Edu03,curso beta,Cartao Nubank.xlsx,2025-05-10
...,...,...,...,...,...,...,...,...,...
494,2025-05-05,NaN,Ultravioleta,49.00,2025-04-05,Imp01,NaN,Cartao Nubank.xlsx,2025-05-10
533,2025-05-05,NaN,Mp *Ojota - Parcela 3/10,200.00,2025-06-12,CPes08,oculos andrea,Cartao Nubank.xlsx,2025-05-10
534,2025-05-05,NaN,Mercadolivre*Iriacaro,89.76,2025-06-12,Laz05,lança teatro lucas,Cartao Nubank.xlsx,2025-05-10
531,2025-05-10,NaN,Apple.Com/Bill,14.90,2025-06-12,Serv07,icloud,Cartao Nubank.xlsx,2025-05-10


In [91]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Credenciais do banco de dados
dbname = "neondb"
user = "neondb_owner"
password = "npg_ayeAjRZ8sPD7"
host = "ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech"
port = "5432"

# Criar a string de conexão com o PostgreSQL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Criar a conexão com o banco de dados
engine = create_engine(connection_string)

In [92]:
from sqlalchemy import text

# Função para excluir dados antigos e inserir novos, com relatórios
def excluir_e_inserir(df, engine):
    with engine.connect() as conn:
        # Verificar se a tabela existe no banco
        check_table_query = text("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'fluxo'
                AND table_name = 'nubank'
            );
        """)
        table_exists = conn.execute(check_table_query).scalar()

        if table_exists:
            # Se a tabela existir, deletar os dados e gravar novamente
            delete_query = text(f"DELETE FROM fluxo.nubank;")
            conn.execute(delete_query)
            conn.commit()
            df.to_sql('nubank', engine, schema='fluxo', if_exists='append', index=False)
        else:
            # Se a tabela não existir, considera que nenhum arquivo foi importado ainda
            df.to_sql('nubank', engine, schema='fluxo', if_exists='append', index=False)

    # Relatório de execução
    print(f"Processamento concluído.")

# Chamar a função para excluir dados antigos e inserir os novos
excluir_e_inserir(df_filtrado, engine)


Processamento concluído.
